# Bài tập
1. Hãy xây dựng mô hình logistic regression bằng tất cả các features trong file heart, so sánh với thư viện sklearn.
2. Hãy xây dựng mô hình softmax regression trên bộ Iris (nên Normalize data), so sánh với thư viện sklearn.

In [1]:
# Import libraries
import pandas as pd
import numpy as np

## Câu 1

In [2]:
def sigmoid(w, x):
    return 1 / (1 + np.exp(-x.dot(w)))


def logistic(w, learning_rate, iterations):
    global X, y

    def grad(w):
        return X.T.dot(sigmoid(w, X) - y).reshape(w.shape[0], -1) / y.shape[0]

    for i in range(iterations):
        w = w - learning_rate * grad(w)
    return w

In [3]:
# Load data
data = pd.read_csv("heart.csv")
data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [4]:
from sklearn.model_selection import train_test_split

# Data
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, :13].values, data.iloc[:, 13].values, test_size=0.33,
                                                    random_state=42)

In [5]:
# Normalize
min0 = np.min(X_train, axis=0)
max0 = np.max(X_train, axis=0)
X_train = (X_train - min0) / (max0 - min0)
X_test = (X_test - min0) / (max0 - min0)
# Add one to X
X = np.concatenate((np.ones((X_train.shape[0], 1)), X_train), axis=1)
y = y_train.reshape(-1, 1)

In [6]:
# Initial w
w = np.zeros([X.shape[1], 1])
w0 = logistic(w, 0.008, 10000)
# Test
X_test_bar = np.concatenate([np.ones([X_test.shape[0], 1]), X_test], axis=1)
a = sigmoid(w0, X_test_bar)
np.mean(np.where(a >= 0.5, 1, 0) == y_test.reshape(-1, 1))

0.8

In [7]:
from sklearn.linear_model import LogisticRegression

# Test by sklearn
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
clf.score(X_test, y_test)

0.81

## Câu 2

In [8]:
def softmax(w, learning_rate, iterations):
    global X, y

    def grad(w):
        w_temp = np.zeros([X.shape[1], y.shape[1]])
        for i in range(y.shape[1]):
            temp = i + 1
            w_temp[:, i:temp] = np.mean(a[:, i:temp] * X, axis=0).reshape(-1, 1)
        return w_temp

    for i in range(iterations):
        expi = np.exp(X.dot(w))
        expsum = np.sum(expi, axis=1)
        soft1 = expi / expsum.reshape(-1, 1)
        a = -y + soft1
        w = w - learning_rate * grad(w)

    return w

In [9]:
from sklearn import datasets

# Load data
iris = datasets.load_iris()

In [10]:
# Data
X0 = iris.data
y0 = iris.target
X_train ,X_test, y_train, y_test = train_test_split(X0, y0, test_size=0.33, random_state=42)

In [11]:
# Normalize
max = np.max(X_train, axis=0)
min = np.min(X_train, axis=0)
X_train = (X_train - min) / (max - min)
X_test = (X_test - min) / (max - min)
# One-hot
y_train_onehot = np.zeros((y_train.size, y_train.max() + 1), dtype=int)
y_train_onehot[np.arange(y_train.size), y_train.reshape(-1)] = 1
y_test_onehot = np.zeros((y_test.size, y_test.max() + 1), dtype=int)
y_test_onehot[np.arange(y_test.size), y_test.reshape(-1)] = 1
# Add one to X
X_train_bar = np.concatenate([np.ones([X_train.shape[0], 1]), X_train], axis=1)
X_test_bar = np.concatenate([np.ones([X_test.shape[0], 1]), X_test], axis=1)
X = X_train_bar
y = y_train_onehot

In [12]:
# Initial w
w = np.zeros([X.shape[1], 3])
w0 = softmax(w, 0.008, 10000)
# Test
expi = np.exp(X_test_bar.dot(w0))
expsum = np.sum(expi, axis=1)
np.mean(y_test.reshape(-1) == np.argmax(expi / expsum.reshape(-1, 1), axis=1))

0.88

In [13]:
# Test by sklearn
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
clf.score(X_test, y_test)

0.9